In [ ]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')

In [ ]:
import torch
from torch import nn
from pytorch_lightning import LightningModule
import pytorch_lightning as pl
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS
from training import lyrics_finetune
from dataclasses import dataclass, asdict
import json
import argparse
from dataclasses import dataclass

In [ ]:
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))


In [ ]:
import flash
import torchaudio
import pytorch_lightning as pl

# class SpeechDataModule(pl.LightningDataModule):
#     def __init__(self, args: WAV2VEC2_ARGS):
#         super().__init__()
#         self.batch_size = 2

#     def prepare_data(self):
#         # Download and extract the audio files and transcripts, if necessary.
#         pass

#     def setup(self, stage=None):
#         # Create the dataset
#         # datamodule = 
#         self.dataset = SpeechRecognitionData.from_csv("consolidated_file_path",
#                                                          "transcription_capitalized",
#                                                          train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
#                                                          test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
#                                                          batch_size=2
#                                                          )

#         # Create the dataloader
#         self.train_dataloader = self.dataset.train_dataloader
#         # torch.utils.data.DataLoader(
#         #     self.dataset,
#         #     batch_size=self.batch_size,
#         #     shuffle=True,
#         #     num_workers=4,
#         # )
#         self.val_dataloader = self.dataset.val_dataloader
#         # torch.utils.data.DataLoader(
#         #     self.dataset,
#         #     batch_size=self.batch_size,
#         #     shuffle=False,
#         #     num_workers=4,
#         # )

#     def train_dataloader(self):
#         return self.train_dataloader

#     def val_dataloader(self):
#         return self.val_dataloader

In [ ]:
# class DALIDataset(pl.LightningDataModule):
#     def __init__(self, batch_size: int = 4,
#                   train_path :Optional[str] = None,
#                     validation_path: Optional[str] = None,
#                       model_backbone: pl.LightningModule = None,
#                       args: TrainingArgs = WAV2VEC2_ARGS
#                       ):
        
#         super().__init__()
#         self.train_path = train_path if train_path is not None else args.TRAIN_FILE_PATH
#         self.validation_path = validation_path if validation_path is not None else args.TEST_FILE_PATH
#         self.model_backbone = model_backbone if model_backbone is not None else args.MODEL_BACKBONE

#         def prepare_data(self):
#             pass
        
#         def setup(self):
#             train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH) 
#             validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
#             songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
#             audio_dataset = Dataset.from_dict(
#                 {"audio": list(songs_metadata["file_name"]),
#                  "transcription": list(songs_metadata["transcription"])}).cast_column("audio", Audio(sampling_rate=16_000))
#             audio_dataset["transcription"] = audio_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', audio_dataset["transcription"]).upper()
#             audio_dataset = audio_dataset.train_test_split(test_size=0.2, shuffle=True)
            



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

# class SpeechRecognitionDataset(Dataset):
#     def __init__(self, dataset,args:TrainingArgs,processor):
#         self.model_backbone = args.MODEL_BACKBONE
#         self.dataset = dataset
#         self.processor = processor
#         self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(self.model_backbone)
#         self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(self.model_backbone)
#         self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        
#     def __getitem__(self, index):
#         audio = self.dataset[index]["audio"][0]
#         transcription = self.dataset[index]["transcription"][0]

#         input_values = self.processor(audio["array"], sampling_rate=16_000).input_values[0]
#         with self.processor.as_target_processor():
#             print("Entering the label encoder")
#             labels = self.processor(transcription,return_tensors = 'pt').input_ids
#        # attention_mask = inputs.attention_mask.squeeze()
#         return input_values, labels
    
#     def __len__(self):
#         return len(self.dataset)



In [ ]:
# from typing import Union, List, Dict
# import torch

from dataclasses import dataclass, field
# from typing import Any, Dict, List, Optional, Union


# class DataCollatorCTCWithPadding:
#     """
#     Data collator that will dynamically pad the inputs received.
#     Args:
#         processor (:class:`~transformers.Wav2Vec2Processor`)
#             The processor used for proccessing the data.
#         padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
#             Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
#             among:
#             * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
#               sequence if provided).
#             * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
#               maximum acceptable input length for the model if that argument is not provided.
#             * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
#               different lengths).
#         max_length (:obj:`int`, `optional`):
#             Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
#         max_length_labels (:obj:`int`, `optional`):
#             Maximum length of the ``labels`` returned list and optionally padding length (see above).
#         pad_to_multiple_of (:obj:`int`, `optional`):
#             If set will pad the sequence to a multiple of the provided value.
#             This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
#             7.5 (Volta).
#     """

#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     max_length_labels: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
#     pad_to_multiple_of_labels: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need
#         # different padding methods
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 max_length=self.max_length_labels,
#                 pad_to_multiple_of=self.pad_to_multiple_of_labels,
#                 return_tensors="pt",
#             )

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

In [ ]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased")
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [ ]:
from typing import Union, Dict, List
from dataclasses import dataclass, field

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        label_attention_features =[{"input_ids": feature["label_attention_mask"]} for feature in features]
        attention_features =[{"input_values": feature["attention_mask"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt")

        batch_attention = self.processor.pad(
            attention_features,
            padding=self.padding,
            return_tensors="pt")
    
            
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        with self.processor.as_target_processor():
            label_attention_batch = self.processor.pad(
                label_attention_features,
                padding=self.padding,
                return_tensors="pt",
            )
                
        # replace padding with -100 to ignore loss correctly
        labels_attention = label_attention_batch["input_ids"].masked_fill(labels_batch.input_ids.eq(101), 0).masked_fill(labels_batch.input_ids.eq(102), 0)
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        

        batch["labels"] = labels
        batch["label_attention_masks"] = labels_attention
        batch["attention_mask"] = batch_attention["input_values"]
        return batch


In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,BertTokenizer
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, AutoModel,AutoProcessor

from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

WAV2VEC2_ARGS.BATCH_SIZE = 2
class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.save_hyperparameters()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.wav2vec2_model)
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
        #self.feature_extractor = AutoFeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        #self.processor = AutoProcessor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        #self.processor = WhisperProcessor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        self.data_collator = DataCollatorCTCWithPadding(processor=self.processor, padding=True)
        self.cpu_count = 4
        print(f"CPU Count = {self.cpu_count}")
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(10)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.train_dataset = train_dataset.map(self.prepare_dataset,remove_columns = train_dataset.column_names)
            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.val_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            test_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            test_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            self.test_dataset = test_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )

    def prepare_dataset(self,batch):
        audio = batch["audio"]
        # batched output is "un-batched" to ensure mapping is correct
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["attention_mask"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).attention_mask[0]
        batch["input_length"] = len(batch["input_values"])
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(batch["transcription"]).input_ids
            batch["label_attention_mask"] = self.processor(batch["transcription"]).attention_mask
        return batch

In [ ]:
#data.val_dataset[0]

In [ ]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, AutoModel

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="mean")
        self.wav2vec2.eval()
        #self.wav2vec2 = AutoModel.from_pretrained(hparams.wav2vec2_model)
        #self.wav2vec2.eval()
        print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.seq2seq.config.is_decoder = True
        self.seq2seq.add_cross_attention = True
        print(self.seq2seq.config)

    def forward(self, audio, attention_mask, labels, label_attention_mask):
        #print("entering forward step")
        encoder_outputs = self.wav2vec2(audio[0],
                                        attention_mask=attention_mask, #Made a change here by adding a [0]
                                        output_hidden_states=True,
                                        output_attentions=True)
        encoder_hidden_states = encoder_outputs[0]  
        print(f"encoder_hidden_states={encoder_hidden_states}")
        encoder_attention_mask = self.wav2vec2._get_feature_vector_attention_mask(
                encoder_hidden_states.shape[1], attention_mask
            )
        print(f"[PRIOR]decoder_input_ids={labels}")
        print(f"[PRIOR]decoder_attention_masks={label_attention_mask}")
        decoder_input_ids = self.shift_tokens_right(labels, 0, 101)
        decoder_attention_masks = self.shift_tokens_right_mask(label_attention_mask)
        print(f"[AFTER]decoder_input_ids={decoder_input_ids}")
        print(f"[AFTER]decoder_attention_masks={decoder_attention_masks}")
        decoder_outputs = self.seq2seq(input_ids=decoder_input_ids,
                                       attention_mask = decoder_attention_masks,
                                       encoder_hidden_states=encoder_hidden_states,
                                      encoder_attention_mask=encoder_attention_mask) #,attention_mask = x.attentions, decoder_input_ids=predicted_ids
        return decoder_outputs

    def generate(self, audio):
        encoder_outputs = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        encoder_hidden_states = encoder_outputs[0]
        bos_ids = (
            torch.ones(
                (encoder_hidden_states.size()[0], 1),
                dtype=torch.long,
                device=self.seq2seq.device,
            )
            * self.seq2seq.config.pad_token_id
        )
        return self.seq2seq.generate(
            input_ids=bos_ids,
            encoder_hidden_states=encoder_hidden_states,
        )

    def training_step(self, batch, batch_idx):
        #print("entering training step")
        audio = batch["input_values"].unsqueeze(0)
        attention_mask = batch["attention_mask"]
        label_attention_mask = batch["label_attention_masks"].unsqueeze(0)
        #print(f"audio={audio}, attention_mask = {attention_mask}, label_attention_mask = {label_attention_mask}")
        labels = batch["labels"]
        labels = labels.reshape(self.batch_size,-1)
        logits = self(audio,attention_mask,labels,label_attention_mask).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        # ctc_loss =  nn.CTCLoss(blank=0)
        # loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        ce_loss = nn.CrossEntropyLoss()
        loss = ce_loss(logits, labels)
        return loss

    def validation_step(self, batch,batch_idx):
        #print("entering validation step")
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        attention_mask = batch["attention_mask"]
        label_attention_mask = batch["label_attention_masks"].unsqueeze(0)
        #print(f"audio={audio}, attention_mask = {attention_mask}, label_attention_mask = {label_attention_mask}")
        labels = labels.reshape(self.batch_size,-1)
        logits = self(audio,attention_mask,labels,label_attention_mask).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        # ctc_loss =  nn.CTCLoss(blank=0)
        # loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        ce_loss = nn.CrossEntropyLoss()
        loss = ce_loss(logits, labels)
        predicted_ids = torch.argmax(logits, dim=-1)
        label_decoded = labels.type(torch.int64).tolist()
        #print(f"labels = {labels}")
        #print(f"predicted ids = {predicted_ids}")
        print(f"original text = {self.bert_tokenizer.decode(label_decoded[0])},{self.bert_tokenizer.decode(label_decoded[1])}")
        predicted_text = predicted_ids.type(torch.int64)
        print(f"Predicted text = {self.bert_tokenizer.decode(predicted_text[:,0].flatten().tolist())},{self.bert_tokenizer.decode(predicted_text[:,-1].flatten().tolist())}")
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

    @staticmethod
    def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
        """
        Shift input ids one token to the right.
        """
        shifted_input_ids = input_ids.new_zeros(input_ids.shape)
        shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
        if decoder_start_token_id is None:
            raise ValueError("Make sure to set the decoder_start_token_id attribute of the model's configuration.")
        shifted_input_ids[:, 0] = decoder_start_token_id
    
        if pad_token_id is None:
            raise ValueError("Make sure to set the pad_token_id attribute of the model's configuration.")
        # replace possible -100 values in labels by `pad_token_id`
        shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    
        return shifted_input_ids

    @staticmethod
    def shift_tokens_right_mask(input_ids: torch.Tensor):
        """
        Shift input ids one token to the right.
        """
        shifted_input_ids = input_ids.new_zeros(input_ids.shape)
        shifted_input_ids[:,:, 1:] = input_ids[:,:, :-1].clone()
        shifted_input_ids[:,:, 0] = 0
        return shifted_input_ids.squeeze()

In [ ]:
import argparse
import pytorch_lightning as pl
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-large-960h-lv60-self' #'openai/whisper-base'# #'openai/whisper-base'
hparams.lm_model = 'bert-base-uncased' #'facebook/bart-large'
hparams.vocab_size = 30000
hparams.learning_rate = 0.00001
hparams.batch_size = WAV2VEC2_ARGS.BATCH_SIZE

model = Wav2SeqModel(hparams)
trainer = Trainer(max_epochs=10,devices=1, accelerator="gpu")
trainer.fit(model,SpeechRecognitionDataModule(WAV2VEC2_ARGS,num_workers=4))

In [ ]:
y_hat

In [ ]:
audio_tensor = next(iter(test_data.val_dataloader()))["input_values"]
audio_tensor

In [ ]:
model.eval()
with torch.no_grad():
    y_hat = model.generate(audio_tensor.unsqueeze(0))
torch.argmax(y_hat, dim=-1)

In [ ]:
# import librosa
# import torch
# from transformers import Wav2Vec2ForCTC

# def create_audio_tensor(audio_path):
#     audio, sample_rate = librosa.load(audio_path)
#     audio_tensor = torch.from_numpy(audio).float()
#     audio_tensor = audio_tensor.unsqueeze(0)
#     return audio_tensor

# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
logits = model(audio_tensor.unsqueeze(0),torch.Tensor(18, 11,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  6,  0,  0,  0,  4,  4,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  0,  0,  0,
          0, 15, 15,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  4,  4,  0,  0,  0,
         14,  0,  0,  0,  0, 10,  0,  0, 12, 12,  0,  0,  0,  0,  0, 19,  0,  0,
          0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 11,
          0,  0,  5,  0,  0,  0,  4,  4,  0,  0,  0, 24,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  5,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  4,  4,  4,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0))
predictions = torch.argmax(logits, dim=-1)
print(audio_tensor)

In [ ]:
predictions

In [ ]:
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

In [ ]:
input,output

In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,BertTokenizer
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, AutoModel,AutoProcessor

from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.save_hyperparameters()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.wav2vec2_model)
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        #self.feature_extractor = AutoFeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        #self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        #self.processor = AutoProcessor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        self.processor = WhisperProcessor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        self.data_collator = DataCollatorCTCWithPadding(processor=self.processor, padding=True)
        self.cpu_count = 4
        print(f"CPU Count = {self.cpu_count}")
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(10)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.train_dataset = train_dataset.map(self.prepare_dataset,remove_columns = train_dataset.column_names)
            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.val_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            test_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            test_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            self.test_dataset = test_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )

    def prepare_dataset(self,batch):
        audio = batch["audio"]
        # batched output is "un-batched" to ensure mapping is correct
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["attention_mask"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).attention_mask[0]
        batch["input_length"] = len(batch["input_values"])
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(batch["transcription"]).input_ids
            batch["label_attention_mask"] = self.processor(batch["transcription"]).attention_mask
        return batch

In [ ]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, AutoModel

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="mean")
        #self.wav2vec2.freeze_feature_encoder()
        self.wav2vec2 = AutoModel.from_pretrained(hparams.wav2vec2_model)
        self.wav2vec2.eval()
        print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.seq2seq.config.is_decoder = True
        self.seq2seq.add_cross_attention = True
        print(self.seq2seq.config)

    def forward(self, audio, attention_mask, labels, label_attention_mask):
        #print("entering forward step")
        encoder_outputs = self.wav2vec2(audio[0],
                                        attention_mask=attention_mask, #Made a change here by adding a [0]
                                        output_hidden_states=True,
                                        output_attentions=True)
        encoder_hidden_states = encoder_outputs[0]  
        print(f"encoder_hidden_states={encoder_hidden_states}")
        encoder_attention_mask = self.wav2vec2._get_feature_vector_attention_mask(
                encoder_hidden_states.shape[1], attention_mask
            )
        #print(f"[PRIOR]decoder_input_ids={labels}")
        #print(f"[PRIOR]decoder_attention_masks={label_attention_mask}")
        decoder_input_ids = self.shift_tokens_right(labels, 0, 101)
        decoder_attention_masks = self.shift_tokens_right_mask(label_attention_mask)
        #print(f"[AFTER]decoder_input_ids={decoder_input_ids}")
        #print(f"[AFTER]decoder_attention_masks={decoder_attention_masks}")
        decoder_outputs = self.seq2seq(input_ids=decoder_input_ids,
                                       attention_mask = decoder_attention_masks,
                                       encoder_hidden_states=encoder_hidden_states,
                                      encoder_attention_mask=encoder_attention_mask) #,attention_mask = x.attentions, decoder_input_ids=predicted_ids
        return decoder_outputs

    def generate(self, audio):
        encoder_outputs = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        encoder_hidden_states = encoder_outputs[0]
        bos_ids = (
            torch.ones(
                (encoder_hidden_states.size()[0], 1),
                dtype=torch.long,
                device=self.seq2seq.device,
            )
            * self.seq2seq.config.pad_token_id
        )
        return self.seq2seq.generate(
            input_ids=bos_ids,
            encoder_hidden_states=encoder_hidden_states,
        )

    def training_step(self, batch, batch_idx):
        #print("entering training step")
        audio = batch["input_values"].unsqueeze(0)
        attention_mask = batch["attention_mask"]
        label_attention_mask = batch["label_attention_masks"].unsqueeze(0)
        #print(f"audio={audio}, attention_mask = {attention_mask}, label_attention_mask = {label_attention_mask}")
        labels = batch["labels"]
        labels = labels.reshape(self.batch_size,-1)
        logits = self(audio,attention_mask,labels,label_attention_mask).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        return loss

    def validation_step(self, batch,batch_idx):
        #print("entering validation step")
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        attention_mask = batch["attention_mask"]
        label_attention_mask = batch["label_attention_masks"].unsqueeze(0)
        #print(f"audio={audio}, attention_mask = {attention_mask}, label_attention_mask = {label_attention_mask}")
        labels = labels.reshape(self.batch_size,-1)
        logits = self(audio,attention_mask,labels,label_attention_mask).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        predicted_ids = torch.argmax(logits, dim=-1)
        label_decoded = labels.type(torch.int64).tolist()
        #print(f"labels = {labels}")
        #print(f"predicted ids = {predicted_ids}")
        print(f"original text = {self.bert_tokenizer.decode(label_decoded[0])},{self.bert_tokenizer.decode(label_decoded[1])}")
        predicted_text = predicted_ids.type(torch.int64)
        print(f"Predicted text = {self.bert_tokenizer.decode(predicted_text[:,0].flatten().tolist())},{self.bert_tokenizer.decode(predicted_text[:,-1].flatten().tolist())}")
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

    @staticmethod
    def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
        """
        Shift input ids one token to the right.
        """
        shifted_input_ids = input_ids.new_zeros(input_ids.shape)
        shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
        if decoder_start_token_id is None:
            raise ValueError("Make sure to set the decoder_start_token_id attribute of the model's configuration.")
        shifted_input_ids[:, 0] = decoder_start_token_id
    
        if pad_token_id is None:
            raise ValueError("Make sure to set the pad_token_id attribute of the model's configuration.")
        # replace possible -100 values in labels by `pad_token_id`
        shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    
        return shifted_input_ids

    @staticmethod
    def shift_tokens_right_mask(input_ids: torch.Tensor):
        """
        Shift input ids one token to the right.
        """
        shifted_input_ids = input_ids.new_zeros(input_ids.shape)
        shifted_input_ids[:,:, 1:] = input_ids[:,:, :-1].clone()
        shifted_input_ids[:,:, 0] = 0
        return shifted_input_ids.squeeze()

In [ ]:
train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH)
validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)

In [ ]:
train_df[train_df.transcription_capitalized.str.len()>200]

# Readying the codebase

In [ ]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')
from training import lyrics_finetune
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS

wandb: Currently logged in as: greeshmasmenon (songslyricstranscription). Use `wandb login --relogin` to force relogin


{
    "TRAIN_FILE_PATH": "/scratch/users/gmenon/train_song_metadata_en_demucs_cleaned_filtered_095.csv",
    "TEST_FILE_PATH": "/scratch/users/gmenon/validation_song_metadata_en_demucs_cleaned_filtered_005.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 2,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/scratch/users/gmenon//model_artefacts/wav2vec2_demucs_en_large-960h-lv60-self_freeze_unfreeze_15epochs_adamw.pt",
    "FINETUNE_STRATEGY": [
        "freeze_unfreeze",
        10
    ],
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}
Namespace(batch_size=2, learning_rate=0.001, lm_model='bert-base-uncased', vocab_size=40000, wav2vec2_model='facebook/wav2vec2-large-960h-lv60-self')


/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_finetuned/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using 'facebook/wa

Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-960h-lv60-self",
  "activation_dropout": 0.1,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dr

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

original text = [CLS] so learn from your mistakes [SEP] [PAD] [PAD] [PAD] [PAD],[CLS] i've been connected to the right line [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] the truth to be found [SEP] [PAD] [PAD] [PAD] [PAD] [PAD],[CLS] he said the way myblue eyes shined [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] you leave me once again home alone [SEP],[CLS] while they are in commend [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] a life all mine [SEP] [PAD],[CLS] so i never went back [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] i

Validation: 0it [00:00, ?it/s]

original text = [CLS] so learn from your mistakes [SEP] [PAD] [PAD] [PAD] [PAD],[CLS] i've been connected to the right line [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] the truth to be found [SEP] [PAD] [PAD] [PAD] [PAD] [PAD],[CLS] he said the way myblue eyes shined [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] you leave me once again home alone [SEP],[CLS] while they are in commend [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] a life all mine [SEP] [PAD],[CLS] so i never went back [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] i

Validation: 0it [00:00, ?it/s]

original text = [CLS] so learn from your mistakes [SEP] [PAD] [PAD] [PAD] [PAD],[CLS] i've been connected to the right line [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] the truth to be found [SEP] [PAD] [PAD] [PAD] [PAD] [PAD],[CLS] he said the way myblue eyes shined [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] you leave me once again home alone [SEP],[CLS] while they are in commend [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] a life all mine [SEP] [PAD],[CLS] so i never went back [SEP]
Predicted text = [CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS],[CLS] [CLS] [CLS] [CLS] [CLS] [CLS] [CLS]
original text = [CLS] i

In [ ]:
import argparse
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-large-960h-lv60-self'
hparams.lm_model = 'bert-base-uncased' #'facebook/bart-large'
hparams.vocab_size = 40000
hparams.learning_rate = 1e-6
hparams.batch_size = 2

model,trainer = lyrics_finetune.run(hparams=hparams)

In [ ]:
# https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/BERT2BERT_for_CNN_Dailymail.ipynb

In [ ]:
# data = trainer.datamodule
# data.setup()
# import torch
# torch.Tensor(data.val_dataset['input_values'][0]).unsqueeze(0)
# model.generate(torch.Tensor(data.val_dataset['input_values'][0]).unsqueeze(0).unsqueeze(0))

In [ ]:
import torch
a = torch.Tensor([6506, 3808,1998, 1996,6506,3808,2058,2058,1012])

In [ ]:
a.tolist()[0]

In [ ]:
b = [101, 2054, 2842, 2023, 2071, 1005, 2310, 2042,  102]

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, BertTokenizer

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
bert_tokenizer.decode(a)

In [ ]:
bert_tokenizer.decode(b)

In [ ]:
b  = [6506, 3808, 2003,1998,2042,3808,7316,1999,1012]

In [ ]:
example_2D_list = [[5, 10, 15, 20],
                   [25, 30, 35, 40],
                   [45, 50, 55, 60]]
list_to_tensor = torch.tensor(example_2D_list)
print("Our New 2D Tensor from 2D List is: ", list_to_tensor)

In [ ]:
list_to_tensor

In [ ]:
tensors = torch.Tensor([[6506, 3808],
        [3808, 3808],
        [2003, 1998],
        [1998, 1996],
        [2042, 6506],
        [3808, 3808],
        [7316, 2058],
        [1999, 2058],
        [1012, 1012]]).type(torch.int64)

In [ ]:
list1 = tensors[:,:1].flatten().tolist()
list2 = tensors[:,-1].flatten().tolist()
list1, list2

In [ ]:
bert_tokenizer.decode(list1)

In [ ]:
bert_tokenizer.decode(list2)

In [ ]:
labels = torch.Tensor([[ 101, 2054, 2842, 2023, 2071, 1005, 2310, 2042,  102],
        [ 101, 2562, 2115, 2192, 1999, 3067,  102, -100, -100]])

In [ ]:
labels.tolist()

In [ ]:
bert_tokenizer.decode(labels[0])

In [ ]:
bert_tokenizer.decode(labels[1])

## TESTING HUGGINGFACE's IMPLEMENTATION

In [ ]:
from transformers import SpeechEncoderDecoderModel, AutoProcessor
from datasets import load_dataset
import torch

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-xls-r-300m-en-to-15")
model = SpeechEncoderDecoderModel.from_pretrained("facebook/wav2vec2-xls-r-300m-en-to-15")

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

input_values = processor(ds[0]["audio"]["array"], return_tensors="pt").input_values
# Inference: Translate English speech to German
generated = model.generate(input_values)
decoded = processor.batch_decode(generated, skip_special_tokens=True)[0]
decoded

# Training: Train model on English transcription
labels = processor(text=ds[0]["text"], return_tensors="pt").input_ids

loss = model(input_values, labels=labels).loss
loss.backward()

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor, SpeechEncoderDecoderModel
from datasets import load_dataset

encoder_id = "facebook/wav2vec2-base-960h"  # acoustic model encoder
decoder_id = "bert-base-uncased"  # text decoder

feature_extractor = AutoFeatureExtractor.from_pretrained(encoder_id)
tokenizer = AutoTokenizer.from_pretrained(decoder_id)
# Combine pre-trained encoder and pre-trained decoder to form a Seq2Seq model
model = SpeechEncoderDecoderModel.from_encoder_decoder_pretrained(encoder_id, decoder_id)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# load an audio input and pre-process (normalise mean/std to 0/1)
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_values = feature_extractor(ds[0]["audio"]["array"], return_tensors="pt").input_values

# load its corresponding transcription and tokenize to generate labels
labels = tokenizer(ds[0]["text"], return_tensors="pt").input_ids

# the forward function automatically creates the correct decoder_input_ids
loss = model(input_values).loss
loss.backward()

In [ ]:
tokenizer.cls_token_id,tokenizer.pad_token_id

In [ ]:
input_values

In [ ]:
labels

In [ ]:
import os
import warnings

import torch
from datasets import load_dataset
from datasets.features import Audio
from transformers import SpeechEncoderDecoderModel, Wav2Vec2Processor


In [ ]:

LANG_ID = "ru"
MODEL_ID = "bond005/wav2vec2-mbart50-ru"
SAMPLES = 30

num_processes = 4

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = SpeechEncoderDecoderModel.from_pretrained(MODEL_ID)


In [ ]:

test_dataset = load_dataset("common_voice", LANG_ID, split=f"test[:{SAMPLES}]")

if test_dataset.features['audio'].sampling_rate != 16_000:
    test_dataset = test_dataset.cast_column(
        'audio',
        Audio(sampling_rate=16_000)
    )

audio_data = [test_dataset[i]['audio']['array'] for i in range(SAMPLES)]

processed = processor(audio_data, sampling_rate=16_000,
                      return_tensors="pt", padding='longest')


In [ ]:

with torch.no_grad():
    predicted_ids = model.generate(**processed)

predicted_sentences = processor.batch_decode(
    predicted_ids,
    num_processes=num_processes,
    skip_special_tokens=True
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i, predicted_sentence in enumerate(predicted_sentences):
        print("-" * 100)
        print("Reference: ", test_dataset[i]["sentence"])
        print("Prediction:", predicted_sentence)

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(10)
train_df

,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,01cef35811fd4a3fa63a3ab8bba5430c,/scratch/users/gmenon/wav_clips/separated/htde...,2.447392,right about now i'm fifty fifty,RIGHT ABOUT NOW I'M FIFTY FIFTY
1,0c04880a6eaf4e559fd9b20594eb74cd,/scratch/users/gmenon/wav_clips/separated/htde...,4.137755,let me run my fingers through your hair,LET ME RUN MY FINGERS THROUGH YOUR HAIR
2,01a7c7f2e6db4c68abfbc1218dc9ef76,/scratch/users/gmenon/wav_clips/separated/htde...,2.923696,but you're yesterday's child to me,BUT YOU'RE YESTERDAY'S CHILD TO ME
3,0c8c027ad9374db6b9e6c12dc873be7b,/scratch/users/gmenon/wav_clips/separated/htde...,2.058390,and whisper so softly,AND WHISPER SO SOFTLY
4,08558a8941c04a559a3ea48793ac1a34,/scratch/users/gmenon/wav_clips/separated/htde...,2.038073,have a laugh at the rat race,HAVE A LAUGH AT THE RAT RACE
5,07929e956a9745fa96e91730d8540cd1,/scratch/users/gmenon/wav_clips/separated/htde...,2.702948,aaaa haaa haaa,AAAA HAAA HAAA
6,002fe3610f1b41b59d28546201635043,/scratch/users/gmenon/wav_clips/separated/htde...,3.507075,but that's not how it used to be,BUT THAT'S NOT HOW IT USED TO BE
7,0a4e93435d0746d39ea0814667d6fa3a,/scratch/users/gmenon/wav_clips/separated/htde...,3.020454,and you've seen it all before but the wolf's o...,AND YOU'VE SEEN IT ALL BEFORE BUT THE WOLF'S O...
8,0bde4da0cf2842028bf7d1a712dec1c5,/scratch/users/gmenon/wav_clips/separated/htde...,2.734830,and talk is cheap awhen the story is good,AND TALK IS CHEAP AWHEN THE STORY IS GOOD
9,0851f1ca1594443d897d4a32b4eab788,/scratch/users/gmenon/wav_clips/separated/htde...,2.288141,turned you into someone new,TURNED YOU INTO SOMEONE NEW


In [6]:
validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)
validation_df

,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,0aa835068aa64515b2158c2f0b6513b7,/scratch/users/gmenon/wav_clips/separated/htde...,4.754943,so learn from your mistakes,SO LEARN FROM YOUR MISTAKES
1,089cd84d662e4c50954cfa46c07cae14,/scratch/users/gmenon/wav_clips/separated/htde...,2.759388,i've been connected to the right line,I'VE BEEN CONNECTED TO THE RIGHT LINE
2,127cc2aaff9a44fab21b2a0c4f5c3e5d,/scratch/users/gmenon/wav_clips/separated/htde...,2.668254,the truth to be found,THE TRUTH TO BE FOUND
3,18e03c3980014046a3504ef42fe81ae8,/scratch/users/gmenon/wav_clips/separated/htde...,2.895079,he said the way myblue eyes shined,HE SAID THE WAY MYBLUE EYES SHINED
4,07c8b8c0b0424c1e87389392695a551b,/scratch/users/gmenon/wav_clips/separated/htde...,2.666939,you leave me once again home alone,YOU LEAVE ME ONCE AGAIN HOME ALONE
5,0cb795c9b6e64a4abab5553c2b2ed3e0,/scratch/users/gmenon/wav_clips/separated/htde...,2.742880,while they are in commend,WHILE THEY ARE IN COMMEND
6,0655395364614218be31548245853ca0,/scratch/users/gmenon/wav_clips/separated/htde...,4.697710,a life all mine,A LIFE ALL MINE
7,113bb605eb324aeb8db046b4193424a2,/scratch/users/gmenon/wav_clips/separated/htde...,2.642948,so i never went back,SO I NEVER WENT BACK
8,00ecd4b71c094b959ae492ec138bb555,/scratch/users/gmenon/wav_clips/separated/htde...,3.176508,in you i taste god,IN YOU I TASTE GOD
9,0b5fa72c054e44598ec8aefb76ff855e,/scratch/users/gmenon/wav_clips/separated/htde...,2.465011,could stay a while,COULD STAY A WHILE


In [16]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-large-960h-lv60-self')

In [17]:
[a for a in zip(bert_tokenizer(list(validation_df["transcription_capitalized"])).input_ids,bert_tokenizer(list(validation_df["transcription"])).input_ids)]

[([101,
   156,
   2346,
   149,
   12420,
   2069,
   2249,
   143,
   21564,
   2107,
   19141,
   2069,
   26574,
   9272,
   1592,
   22441,
   1708,
   102],
  [101, 1177, 3858, 1121, 1240, 12572, 102]),
 ([101,
   146,
   112,
   159,
   2036,
   139,
   27073,
   2249,
   18732,
   2249,
   22680,
   16647,
   10069,
   16972,
   7462,
   155,
   23413,
   18784,
   149,
   11607,
   2036,
   102],
  [101, 178, 112, 1396, 1151, 3387, 1106, 1103, 1268, 1413, 102]),
 ([101,
   7462,
   157,
   2069,
   16830,
   3048,
   16972,
   139,
   2036,
   143,
   2346,
   27370,
   2137,
   102],
  [101, 1103, 3062, 1106, 1129, 1276, 102]),
 ([101,
   145,
   2036,
   13411,
   9949,
   7462,
   22751,
   3663,
   150,
   3663,
   13360,
   24846,
   142,
   3663,
   9919,
   17730,
   11607,
   10069,
   102],
  [101, 1119, 1163, 1103, 1236, 1139, 1830, 19224, 1257, 18978, 1181, 102]),
 ([101,
   19141,
   149,
   12420,
   17145,
   22157,
   21748,
   10954,
   14731,
   1592,
   11607

In [11]:
bert_tokenizer.decode(bert_tokenizer(list(validation_df["transcription"])).input_ids[3],skip_special_tokens=True)

'he said the way myblue eyes shined'

In [12]:
list(validation_df["transcription"])[0]

'so learn from your mistakes'

In [4]:
import torch
label = torch.Tensor([101, 155, 23413, 18784, 16151, 2346, 16830, 24819, 2924, 146, 112, 150, 143, 15499, 16880, 143, 15499, 16880, 102])

label = torch.Tensor([[ 101, 1037, 2166, 2035, 3067,  102, -100],[ 101, 2061, 1045, 2196, 2253, 2067,  102]])

In [5]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-base-960h')

In [6]:
bert_tokenizer.decode(label[1])

'[CLS] so i never went back [SEP]'

In [7]:
wav_tokenizer.decode(label)

NameError: name 'wav_tokenizer' is not defined

In [ ]:
label = torch.Tensor([[ 101, 1037, 2166, 2035, 3067,  102, -100],
        [ 101, 2061, 1045, 2196, 2253, 2067,  102]])

In [ ]:
bert_tokenizer.decode(label[:,0])

In [ ]:
bert_tokenizer.decode(label[0,:])

In [ ]:
predicted = torch.Tensor([[1996, 1996],
        [1999, 1996],
        [2061, 1996],
        [2061, 1000],
        [1000, 1998],
        [1000, 2030],
        [2061, 2061]])

In [ ]:
bert_tokenizer.decode(predicted.T[0,:],skip_special_tokens=True)

In [3]:
import pandas as pd
validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)

In [37]:
list(validation_df[validation_df["Folder"] == '127cc2aaff9a44fab21b2a0c4f5c3e5d']["transcription_capitalized"])

['THE TRUTH TO BE FOUND']

In [38]:
bert_tokenizer(list(validation_df[validation_df["Folder"] == '127cc2aaff9a44fab21b2a0c4f5c3e5d']["transcription_capitalized"])).input_ids

[[101, 1996, 3606, 2000, 2022, 2179, 102]]

In [28]:
def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    #batch["attention_mask"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).attention_mask[0]
    batch["input_length"] = len(batch["input_values"])
    batch["labels"] = bert_tokenizer(batch["transcription"]).input_ids
    batch["label_attention_mask"] = bert_tokenizer(batch["transcription"]).attention_mask
    return batch


In [29]:
from datasets import Dataset, Audio
val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

In [51]:
val_dataset1 = val_dataset.map(prepare_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [52]:
#val_dataset1[2]

In [53]:
label_features = [{"input_ids": feature["labels"]} for feature in val_dataset1]

In [50]:
bert_tokenizer.pad(
            label_features,
            padding=True,
            return_tensors="pt",
            )

{'input_ids': tensor([ 101, 1996, 3606, 2000, 2022, 2179,  102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1])}

In [18]:
len(bert_tokenizer.get_vocab())

30522

In [26]:
weights

tensor([0., 0., 0.,  ..., 1., 1., 1.])